Precision and recall
https://insidelearningmachines.com/precisionk_and_recallk/#:~:text=Precision%40k%20and%20Recall%40k%20are%20metrics%20used%20to%20evaluate,end%20user%20by%20the%20model.

In [1]:
# imports
import pandas as pd
import numpy as np
from typing import List
import os
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings


In [2]:
interactions = pd.read_csv("MIND/behaviors.tsv",sep='\t',  header=None)
interactions.columns =['User', 'Time', 'ID', 'Impressions'] 
interactions = interactions.drop(['Time'], axis=1)
interactions.head()

,User,ID,Impressions
0,U13740,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,U91836,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,U73700,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,U34670,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,U8125,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [3]:
#users = interactions_emb
interactions_emb = pd.read_csv("embeddings/users_filtered_final.csv") #document with user interactions
interactions_emb.head()

,User,ID,Interactions_emb
0,U244,N17157 N38621 N35022 N50578 N264 N9120 N23907 ...,"[-0.005149974951877837, -0.013250857458654631,..."
1,U68369,N19381 N54536,"[0.0025621717686590273, 0.004183989018201828, ..."
2,U50236,N4020 N44292 N50292 N40772 N57737 N33969 N4054...,"[-0.010138329240492436, -0.01179651383115145, ..."
3,U77060,N23105 N41375,"[-0.005568941123783588, -0.025914330035448074,..."
4,U5596,N459 N56253 N62931 N55846 N29849 N45729 N62834...,"[-0.012533644353970886, -0.011675744312297967,..."


In [4]:
#load the data with news articles
news = pd.read_csv("embeddings/news_emb_final.csv") #document with news content
news.head()

,ID,Category,SubCategory,Content,Content_emb
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","[0.005885085556656122, -0.007782096974551678, ..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat These seemingly ...,"[-0.004876355174928904, -0.007969613187015057,..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,"[-0.02760046347975731, -0.013719998300075531, ..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"[-0.0297758337110281, -0.014837449416518211, 0..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","[0.005073545966297388, 0.004160495940595865, 0..."


Calculate the proportion of -1 and 0 in 'Impressions' 

In [15]:
# Function to count the number of suffixes
def count_suffixes(row, suffix):
    impressions = row['Impressions'].split()
    count = sum(1 for imp in impressions if imp.endswith(suffix))
    return count

In [16]:
# Counting "-1" and "-0" suffixes
interactions['-1 Count'] = interactions.apply(lambda row: count_suffixes(row, '-1'), axis=1)
interactions['-0 Count'] = interactions.apply(lambda row: count_suffixes(row, '-0'), axis=1)

# Total count across all users
total_minus_1 = interactions['-1 Count'].sum()
total_minus_0 = interactions['-0 Count'].sum()

print("Total -1 count:", total_minus_1)
print("Total -0 count:", total_minus_0)

Total -1 count: 236344
Total -0 count: 5607100


In [17]:
total_minus_1/total_minus_0

0.04215084446505324

Calculate precision and recall

In [7]:
def create_user_df(input_df, user):
    user_row = input_df[input_df['User'] == user]

    if user_row.empty:
        return None

    impressions = user_row['Impressions'].values[0].split()

    news_ids = []
    true_values = []

    for impression in impressions:
        news_id, true_value = impression.split('-')
        news_ids.append(news_id)
        true_values.append(int(true_value))

    user_df = pd.DataFrame({'ID': news_ids, 'true_value': true_values})
    return user_df

In [8]:
def list_ids_in_folder(folder_path):
    ids = set()
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            # Extracting the ID from the filename
            file_id = filename.split("_")[0][1:]
            ids.add(file_id)
    
    return list(ids)

In [9]:
folder_path = 'hybrid_recommendations'
users_list = list_ids_in_folder(folder_path)
users_list = ['U' + num for num in users_list]
len(users_list)

49053

In [34]:
recommendations_hybrid = pd.read_csv("recommendations_20/hybrid_rec_20.csv")

In [36]:
users_list = recommendations_hybrid['User_ID'].unique().tolist()

In [39]:
#recommder_type = [content, collab, hybrid]
def precision_and_recall (list_name, recommender_type):

    
    all_precision = []
    all_recall = []
    
    user_df = list_name
    
    for i in users_list[:10]:
        if recommender_type == 'content':
            column = 'Distance'
            path = "content_recommendations/" + i + "_content.csv"
        if recommender_type == 'collab':
            column = 'article_distance'
            path = "collaborative_recommendations/" + i + "_collab.csv"
        if recommender_type == 'hybrid':
            column = 'hybrid'
            path = "hybrid_recommendations/" + i + "_hybrid.csv"
        
        user_df = create_user_df(interactions, i)
        news = pd.read_csv(path) 
        #new sorting for hybrid
        news = news.sort_values(by='hybrid')
        
        #print (news)
        
        id_to_new_column = news.set_index('ID')[column].to_dict()
        user_df[column] = user_df['ID'].map(id_to_new_column)
        
        three_percent = np.percentile(news[column], 4)
        
        user_df['predicted_value'] = user_df[column].apply(lambda x: 0 if x >= three_percent else 1)
        
        # Assuming df is your DataFrame
        true_values = user_df['true_value']
        predicted_values = user_df['predicted_value']

        # Calculate precision
        precision = precision_score(true_values, predicted_values, average='macro') #'weighted' or 'macro'
        recall =recall_score(true_values, predicted_values, average='macro') #'weighted' or 'macro'
        
        all_precision.append(precision)
        all_recall.append(recall)
        #print(user_df)
    print(np.mean(all_precision))
    print(np.mean(all_recall))

In [11]:
warnings.filterwarnings("ignore")

In [19]:
precision_and_recall(users_list, "hybrid")

           ID    hybrid
221    N36903  0.088961
999    N17040  0.090797
3427   N56248  0.091204
3619   N61837  0.091298
4537   N28856  0.092475
...       ...       ...
50044   N2544  0.982586
49129  N10305  0.994560
49736   N9846  0.995951
39040   N2242  0.999277
48975  N33653  1.018836

[51277 rows x 2 columns]
           ID    hybrid
5480   N28419  0.080995
5506   N11291  0.083885
8636   N56586  0.085757
2706   N40272  0.086323
1287     N997  0.086766
...       ...       ...
41015   N3477  0.987285
49192  N10305  0.999215
39032   N2242  0.999810
49774   N9846  1.000369
49040  N33653  1.013458

[51276 rows x 2 columns]
           ID    hybrid
2362   N53296  0.067126
879    N34470  0.069150
3294   N46064  0.070352
2607    N1304  0.071439
356    N16212  0.071722
...       ...       ...
49905   N2544  0.985885
49742   N9846  0.998288
38907   N2242  0.998700
49164  N10305  1.000003
49401  N33653  1.014500

[51255 rows x 2 columns]
           ID    hybrid
6306   N55805  0.069053
776    N27

In [40]:
from tqdm import tqdm  # Import tqdm for progress bar

def precision_and_recall(list_name, recommender_type):
    all_precision = []
    all_recall = []
    user_df = list_name
    
    for i in tqdm(users_list, desc='Processing users'):  # Progress bar added
        if recommender_type == 'content':
            column = 'Distance'
            path = "content_recommendations/" + i + "_content.csv"
        if recommender_type == 'collab':
            column = 'article_distance'
            path = "collaborative_recommendations/" + i + "_collab.csv"
        if recommender_type == 'hybrid':
            column = 'hybrid'
            path = "hybrid_recommendations/" + i + "_hybrid.csv"
        
        user_df = create_user_df(interactions, i)
        news = pd.read_csv(path) 
        news = news.sort_values(by='hybrid')
        
        id_to_new_column = news.set_index('ID')[column].to_dict()
        user_df[column] = user_df['ID'].map(id_to_new_column)
        
        three_percent = np.percentile(news[column], 4)
        
        user_df['predicted_value'] = user_df[column].apply(lambda x: 0 if x >= three_percent else 1)
        
        true_values = user_df['true_value']
        predicted_values = user_df['predicted_value']

        precision = precision_score(true_values, predicted_values, average='macro')
        recall = recall_score(true_values, predicted_values, average='macro')
        
        all_precision.append(precision)
        all_recall.append(recall)
    
    print("Average Precision:", np.mean(all_precision))
    print("Average Recall:", np.mean(all_recall))


In [41]:
precision_and_recall(users_list, "hybrid")

Processing users:   0%|          | 0/49052 [00:00<?, ?it/s]

Processing users: 100%|██████████| 49052/49052 [4:49:43<00:00,  2.82it/s]  

Average Precision: 0.4549958606944153
Average Recall: 0.5094872228816011
